 #### <u>Lab 10</u>
##### CSCI316 - Big Data

**Group Members**
- Hunter Riddle (6451159)
- Oliver Small ()

#### Plain Tensorflow MLP Implementation

In [1]:
# imports
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings

# ignore TF warnings
tf.logging.set_verbosity(tf.logging.ERROR)

/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/frame

In [2]:
# read in data
data = pd.read_csv("abalone.data", sep=',', header=None)
n_rows = data.shape[0]
n_columns = data.shape[1]

In [3]:
# format data
data.columns = ['sex', 'length', 'diameter', 'height', 'whole_weight', 'shucked_weight', 'viscera_weight', 'shell_weight', 'rings']

# one hot encoding ('sex' attribute)
for (num, value) in enumerate(set(data['sex']), start=0):
    data['sex'] = data['sex'].replace([value], num)

In [4]:
# create testsets (holdout evaluation method, 70/30 split)
X_train = data.sample(int(n_rows*0.7))
X_test = data.drop(X_train.index)

# reset indexes
X_train = X_train.reset_index()
X_test = X_test.reset_index()

# cleaning up
y_train = np.array(X_train['rings'])
y_test = np.array(X_test['rings'])
X_train = np.array(X_train.iloc[: ,1:-1])
X_test = np.array(X_test.iloc[: ,1:-1])

In [5]:
# MLP hyperparameters
n_features = 8
n_neurons_hidden = 10
n_neurons_out = 1

In [6]:
# MLP variables
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
y_true = tf.placeholder(tf.float32, shape=(None), name="y_true")

W1 = tf.get_variable("weights1", dtype=tf.float32, initializer=tf.zeros((n_features, n_neurons_hidden)))
b1 = tf.get_variable("bias1", dtype=tf.float32, initializer=tf.zeros((n_neurons_hidden)))
W2 = tf.get_variable("weights2", dtype=tf.float32, initializer=tf.zeros((n_neurons_hidden, n_neurons_out)))
b2 = tf.get_variable("bias2", dtype=tf.float32, initializer=tf.zeros((n_neurons_out)))

In [7]:
# create network
h = tf.nn.sigmoid(tf.matmul(X, W1) + b1)
z = tf.matmul(h, W2) + b2

In [ ]:
# define costs
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=z, labels=y_true)
cost = tf.reduce_mean(cross_entropy)

# define accuracy
#accuracy = tf.metrics.accuracy(labels=tf.squeeze(set(y_test)), predictions=tf.argmax(z, axis=1),)[1]

In [ ]:
# training
learning_rate = 0.1
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_op = optimizer.minimize(cost)

In [ ]:
# execution
init = tf.global_variables_initializer()
n_epochs = 40
with tf.Session() as session:
    init.run()
    for epoch in range(n_epochs):
        session.run(training_op, feed_dict={
            X: X_train,
            y_true: y_train
        })
        
    predictions = session.run(z, feed_dict={'X':X_test})
    print(predictions[33:57])
   